# Data Aggregation and Group Operations

## GroupBy Mechanics

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.054922,0.277862
1,a,two,1.297727,1.401855
2,b,one,0.709605,-0.027009
3,b,two,0.545099,0.858209
4,a,one,0.732454,1.606900


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a    1.028368
b    0.627352
Name: data1, dtype: float64

In [6]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     one     0.893688
      two     1.297727
b     one     0.709605
      two     0.545099
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.893688,1.297727
b,0.709605,0.545099


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [9]:
df["data1"].groupby([states, years]).mean()

California  2005    1.297727
            2006    0.709605
Ohio        2005    0.800010
            2006    0.732454
Name: data1, dtype: float64

If the grouping information is found in the same DataFrame as the data we want to work with. Column names are passed as the group keys.

In [10]:
df.groupby("key1").mean()

,data1,data2
key1,,
a,1.028368,1.095539
b,0.627352,0.415600


In [11]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one   0.893688  0.942381
     two   1.297727  1.401855
b    one   0.709605 -0.027009
     two   0.545099  0.858209

In [12]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [13]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.054922  0.277862
1    a  two  1.297727  1.401855
4    a  one  0.732454  1.606900
b
  key1 key2     data1     data2
2    b  one  0.709605 -0.027009
3    b  two  0.545099  0.858209


In [14]:
for (k1,k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.054922  0.277862
4    a  one  0.732454  1.606900
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.297727  1.401855
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.709605 -0.027009
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.545099  0.858209


By default **groupby** groups on axis=0, but it can be done on any of the other axes.

In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [16]:
grouped = df.groupby(df.dtypes, axis=1)

In [17]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.054922  0.277862
1  1.297727  1.401855
2  0.709605 -0.027009
3  0.545099  0.858209
4  0.732454  1.606900
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [18]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    one   0.942381
     two   1.401855
b    one  -0.027009
     two   0.858209

In [19]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [20]:
s_grouped.mean()

key1  key2
a     one     0.942381
      two     1.401855
b     one    -0.027009
      two     0.858209
Name: data2, dtype: float64

### Grouping with Dict and Series

In [22]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-1.081424,-0.763737,0.858730,0.382697,-0.654667
Steve,-0.531522,-0.824707,0.908576,-1.604550,2.062606
Wes,1.574660,-1.898837,1.724030,0.835699,-1.906153
Jim,0.941715,-0.649579,-0.838223,-0.772149,-0.369076
Travis,1.417807,-2.371869,0.989955,0.767078,0.314093


In [23]:
# Add a few NA values
people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,-1.081424,-0.763737,0.858730,0.382697,-0.654667
Steve,-0.531522,-0.824707,0.908576,-1.604550,2.062606
Wes,1.574660,NaN,NaN,0.835699,-1.906153
Jim,0.941715,-0.649579,-0.838223,-0.772149,-0.369076
Travis,1.417807,-2.371869,0.989955,0.767078,0.314093


In [24]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [26]:
by_column = people.groupby(mapping, axis=1)
print(by_column)

In [27]:
by_column.sum()

,blue,red
Joe,1.241428,-2.499828
Steve,-0.695974,0.706376
Wes,0.835699,-0.331494
Jim,-1.610372,-0.076940
Travis,1.757034,-0.639969


In [28]:
map_series = pd.Series(mapping)

In [29]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [31]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.434951,-1.413317,0.020508,0.446247,-2.929896
5,-0.531522,-0.824707,0.908576,-1.604550,2.062606
6,1.417807,-2.371869,0.989955,0.767078,0.314093


In [32]:
key_list = ["one", 'one', "one", "two", "two"]

In [33]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.081424 -0.763737  0.858730  0.382697 -1.906153
  two  0.941715 -0.649579 -0.838223 -0.772149 -0.369076
5 one -0.531522 -0.824707  0.908576 -1.604550  2.062606
6 two  1.417807 -2.371869  0.989955  0.767078  0.314093

### Grouping by Index Levels

In [35]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [36]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)

In [37]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.968976 -0.252069 -0.841185 -0.394773 -0.332959
1      1.231062 -0.012215  0.213305  0.080502 -0.308163
2     -0.036813  0.578589  1.083302  1.799488  0.089196
3      1.287484  0.255071 -1.526140 -0.589999 -2.559221

## Data Aggregation

In [38]:
df

,key1,key2,data1,data2
0,a,one,1.054922,0.277862
1,a,two,1.297727,1.401855
2,b,one,0.709605,-0.027009
3,b,two,0.545099,0.858209
4,a,one,0.732454,1.606900


In [39]:
grouped = df.groupby("key1")
grouped

In [40]:
grouped["data1"].quantile(0.9)

key1
a    1.249166
b    0.693154
Name: data1, dtype: float64

To use own aggregation functions, pass any function that aggregates an array to the **aggregate** or **agg** method.

In [41]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.565274,1.329038
b,0.164506,0.885217


In [42]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.028368  0.283571  0.732454  0.893688  1.054922  1.176325   
b      2.0  0.627352  0.116323  0.545099  0.586225  0.627352  0.668478   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.297727   3.0  1.095539  0.715512  0.277862  0.839859  1.401855   
b     0.709605   2.0  0.415600  0.625943 -0.027009  0.194296  0.415600   

                          
           75%       max  
key1                      
a     1.504377  1.606900  
b     0.636904  0.858209

### Column-Wise and Multiple Function Application